# Main Testing Notebook

This notebook contains the main testing procedures of the project.

In [22]:
import os
import numpy as np
import open3d as o3d
from descriptors import DescriptorWrapper

In [30]:
def load_mesh_and_sample_points(file_name, num_points=5000):
    # Default number of points as indicated in the thesis
    mesh = o3d.io.read_triangle_mesh(file_name)
    point_cloud = mesh.sample_points_poisson_disk(num_points)
    return np.asarray(point_cloud)

In [ ]:
path = 'mesh_files'

mesh_objects = []
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    if os.path.isfile(file_path):
        object_category = ''
        if 'airplane' in file:
            object_category = 'airplane'
        elif 'dino' in file:
            object_category = 'dinosaur'
        else: 
            object_category = 'fish'
        mesh_objects.append([load_mesh_and_sample_points(file_path), object_category])

In [ ]:
descriptor_wrapper = DescriptorWrapper()